In [39]:
import os
import re
import pandas as pd
from IPython.display import display
import json

directory_root = "E:\Programs\AI Finetuning\Subs"
directory_raw = fr"{directory_root}\Raw" + "\\"
directory_cleaned = fr"{directory_root}\Cleaned" + "\\"

In [47]:
data = []
for filename in os.listdir(directory_raw):
    if filename.endswith(".ass"):
        episode = int(re.search(r'\d+', filename).group())
        with open(os.path.join(directory_raw, filename), "r", encoding="utf-8") as f:
            lines = f.readlines()
            event_start = lines.index("[Events]\n") + 2

            prev_name, prev_start, prev_end, prev_text = None, None, None, ""

            for line in lines[event_start:]:
                if line.startswith("Dialogue:"):
                    fields = line.split(",", 9)
                    layer, start, end, style, name, _, _, _, _, text = fields

                    if style == "On Screen":
                        continue

                    # Carry over the previous name if the current name is empty
                    if not name.strip():
                        name = prev_name

                    # Remove formatting tags from text
                    text = re.sub(r"{[^}]+}", "", text).strip()

                    if prev_name == name:
                        prev_end = end
                        prev_text += " " + text
                    else:
                        if prev_name is not None:
                            data.append([episode, prev_start, prev_end, prev_name, prev_text])

                        prev_name, prev_start, prev_end, prev_text = name, start, end, text

            # Add the last dialogue line
            if prev_name is not None:
                data.append([episode, prev_start, prev_end, prev_name, prev_text])

# Create a DataFrame
df = pd.DataFrame(data, columns=["Episode", "Start", "End", "Name", "Text"])
df.drop(columns=["Start", "End"], inplace=True)

In [86]:
# Filter the DataFrame to exclude rows containing "caption" or "episode title" in the Name column
filtered_df = df.loc[~df['Name'].str.contains("caption|episode title|EpTitle", case=False, na=False)].copy()
# Fix line formatting text
filtered_df["Text"] = filtered_df.loc[:,'Text'].str.replace(r"\\N", "", case=False, regex=True)

best_episodes = [20,25,29,34,37,38,45,46,48,50,53,54,55,66,69,72,73,79,88,89,90,91,92,94,95,96,98,99,100,109,110,111,114,119,120,121,122,123,125,131,132,133,134,150,154,155,165,166,167,168,169,170,182,183,184,202,203,216,218,225,226,228,229,237,238,239,240]
filtered_df = filtered_df[filtered_df["Episode"].isin(best_episodes)]

print(len(best_episodes))

# display(filtered_df)

# Specific name mappings dictionary
name_mappings = {
    "Gin": "GINTOKI",
    "Shin": "SHINPACHI",
    "Zura": "KATSURA",
    "MADAO": "HASEGAWA",
    "KYU": "KYUBEI",
    "Gura": "KAGURA"
}

unique_names = set(filtered_df["Name"])
unique_names_without_m = {name.rstrip("M") for name in unique_names}

def adjust_name(name):
    lower_name = name.lower()
    
    for key, value in name_mappings.items():
        if lower_name == key.lower():
            return value
    

    name = re.sub(r'[^a-zA-Z0-9\s\']', '', name)  # Keep alphanumeric characters, spaces, and apostrophes
    name = name.strip()

    # Remove the trailing 'M' if the name without 'M' is in the list of unique names
    if name.endswith(" M") and name[:-2] in unique_names_without_m and not name.endswith("MOM"):
        name = name[:-2]

    # Remove the trailing ' MONO' if the name without ' MONO' is in the list of unique names
    if name.upper().endswith(" MONO") and name.upper()[:-5] in unique_names:
        name = name[:-5]
    return name.upper()

# Apply name adjustments
filtered_df["Name"] = filtered_df["Name"].apply(adjust_name)

# Group by 'Name' and count the number of dialogue items
name_dialogue_count = filtered_df.groupby('Name')['Text'].count()

# Convert the Pandas Series to a dictionary
name_dialogue_count_dict = name_dialogue_count.to_dict()

# Sort the dictionary by values in descending order
sorted_name_dialogue_count = dict(sorted(name_dialogue_count_dict.items(), key=lambda x: x[1], reverse=True))

print("Sorted dictionary of names and their dialogue counts:")
# for name, count in sorted_name_dialogue_count.items():
#     print(f"{name}: {count}")

display(filtered_df)

67
Sorted dictionary of names and their dialogue counts:


Episode       Name                                               Text
3165        20    INAYAMA  It was a very hot night with many mosquitoes, ...
3166        20   HIJIKATA                      We've run out of mayonnaise!!
3167        20  SAMURAI A  Vice-chief!! What the heck-?! You ruined the e...
3168        20   HIJIKATA  I don't care about that! The mayonnaise ran ou...
3169        20  SAMURAI B  You've had enough, already! That doesn't look ...
...        ...        ...                                                ...
39805      240       BOTH                                               Yes!
39806      240    GINTOKI                             Attention, all humans!
39807      240       BOTH                               Thank you very much!
39808      240    GINTOKI                                      Male escorts!
39809      240        ALL                                        Just do it!

[12046 rows x 3 columns]

In [87]:
# Set of names you want to search for
# names = {'GINTOKI'}
names = {'KAGURA'}

lines_before_dialogue = 5
lines_after_dialogue = 3

# Create an additional column to store True/False values based on nearby rows
filtered_df['keep'] = False

# Iterate over unique episodes and check for the names within a range of rows
for episode in filtered_df['Episode'].unique():
    episode_df = filtered_df[filtered_df['Episode'] == episode]
    
    # Check lines where Gintoki's dialogue is present
    filtered_df.loc[episode_df.index, 'keep'] |= episode_df['Name'].isin(names)

    # Check lines before dialogue
    for i in range(1, lines_before_dialogue + 1):
        filtered_df.loc[episode_df.index, 'keep'] |= episode_df['Name'].shift(i).isin(names)

    # Check lines after dialogue
    for i in range(1, lines_after_dialogue + 1):
        filtered_df.loc[episode_df.index, 'keep'] |= episode_df['Name'].shift(-i).isin(names)

# Filter rows based on the 'keep' column
df_filtered = filtered_df[filtered_df['keep']]

# Filter rows based on the 'keep' column
df_filtered2 = filtered_df[filtered_df['keep']].copy()

# Reset the index and drop the 'keep' column
df_filtered2.reset_index(drop=True, inplace=True)
df_filtered2.drop(columns=['keep'], inplace=True)

# Save the filtered dataframe to a new CSV file
df_filtered2.to_csv("filtered_data_kagura.csv", index=False)

display(df_filtered2)


Episode       Name                                               Text
0          20   HIJIKATA         Who are these guys? Is the circus in town?
1          20      KONDO                 No, they're here for a exorcising.
2          20   HIJIKATA  Hey, hey! Are you kidding? These guys look so ...
3          20     KAGURA                                   Oh? Your back...
4          20   HIJIKATA             What...? There's something on my back?
...       ...        ...                                                ...
4335      239  SHINPACHI                            I hope nothing happens.
4336      239    GINTOKI  Ow... What happened yesterday? No good. Can't ...
4337      239      OTOSE        R-Really. I can't remember anything either.
4338      239    GINTOKI  Why isn't she looking at me? Why is she acting...
4339      239      OTOSE  That's perfect. Neither of us remembers. Nothi...

[4340 rows x 3 columns]

In [89]:
# Chunk dialogue into prompt: completion + make prompt blank

# episode_list = [i for i in range(3, 245)]

# print(episode_list)

relevant_episode_list = []  # use to not summarize wiki later?
prompt_completion_list = []

# temp_list = [20]

for episode in best_episodes: #  # temp_list
    current_episode_df = df_filtered2[df_filtered2["Episode"] == episode]

    num_dialogue = current_episode_df.shape[0]
    if num_dialogue != 0:
        relevant_episode_list.append(episode)

        dialogue_segments = []

        for idx, row in current_episode_df.iterrows():
            # if row['Name'] == 'GINTOKI':
            if row['Name'] == 'KAGURA':
                start_idx = max(current_episode_df.index[0], idx - lines_before_dialogue)
                end_idx = idx

                segment_df = current_episode_df.loc[start_idx:end_idx].copy()
                segment_df_prompt = segment_df.copy(deep=True)

                last_gintoki_idx = segment_df_prompt[segment_df_prompt['Name'] == 'KAGURA'].index[-1]
                segment_df_prompt.at[last_gintoki_idx, "Text"] = "<mask>"

                segment = {"prompt": segment_df_prompt, "completion": segment_df.at[last_gintoki_idx, "Text"]}
                # print(segment["completion"])
                dialogue_segments.append(segment)

        prompt_completion_list.append(dialogue_segments)

print(relevant_episode_list)
print("Example dialogue segment:")
display(prompt_completion_list[0][0])




[20, 25, 29, 34, 37, 38, 45, 46, 48, 50, 53, 54, 55, 66, 69, 72, 73, 79, 88, 89, 90, 91, 92, 94, 95, 98, 99, 100, 111, 114, 120, 121, 122, 123, 125, 131, 132, 133, 134, 150, 154, 155, 165, 167, 168, 169, 170, 182, 183, 184, 202, 203, 216, 218, 225, 228, 229, 237, 238, 239]
Example dialogue segment:


{'prompt':    Episode      Name                                               Text
 0       20  HIJIKATA         Who are these guys? Is the circus in town?
 1       20     KONDO                 No, they're here for a exorcising.
 2       20  HIJIKATA  Hey, hey! Are you kidding? These guys look so ...
 3       20    KAGURA                                             <mask>,
 'completion': 'Oh? Your back...'}

In [90]:
display(prompt_completion_list[0][2])
print(len(prompt_completion_list[0]))

{'prompt':     Episode     Name                                               Text
 7        20    KONDO  Sensei, can you help us? I'm too scared to go ...
 8        20   KAGURA                    We'll take care of it, gorilla.
 9        20    KONDO  Huh? Did you just say "gorilla"? You just said...
 10       20  GINTOKI  We looked around the house. And I feel a fairl...
 11       20    KONDO     Ah, you definitely said "gorilla," didn't you?
 12       20   KAGURA                                             <mask>,
 'completion': "For starters, why don't we do an exorcism? This is going to cost you a pretty penny, gorilla."}

21


In [85]:
def convert_dataframe_to_text(df, mask_target=None):
    dialogue_list = [{"name": name, "text": text} for name, text in zip(df['Name'], df['Text'])]
    return dialogue_list

def get_episode_context(episode_number):
    # Replace this with the actual function to fetch the episode context.
    return f"Context for episode {episode_number}"

identifier = 1
with open(directory_cleaned + 'finetuning_kagura.jsonl', 'w+') as file:
    for episode_segments in prompt_completion_list:
        for segment in episode_segments:
            episode_number = int(segment['prompt'].iloc[0]['Episode'])  # Convert to int
            context = get_episode_context(episode_number)
            
            prompt_dialogue = convert_dataframe_to_text(segment['prompt'], mask_target='GINTOKI')
            completion_text = segment['completion']

            json_line = json.dumps({
                "prompt": {
                    "identifier": identifier,
                    "episode": episode_number,
                    "context": context,
                    "dialogue": prompt_dialogue
                },
                "completion": {
                    "text": completion_text
                }
            }, ensure_ascii=False)
            identifier += 1

            file.write(json_line + '\n')


In [8]:
df_filtered2.to_csv(directory_cleaned + "All_Dialogue.csv", index=False)

In [9]:
main_relations = list(sorted_name_dialogue_count.keys())[:31]
ignore_relations = ["N", "A", "SA", "ALL", "MAN", "B", "BOTH"]
for people in ignore_relations:
    if people in main_relations:
        main_relations.remove(people)

print(main_relations)

['GINTOKI', 'SHINPACHI', 'KAGURA', 'HIJIKATA', 'KONDO', 'KATSURA', 'OKITA', 'OTAE', 'HASEGAWA', 'YAMAZAKI', 'OTOSE', 'SACHAN', 'KYUBEI', 'TOSHI', 'TSUKUYO', 'ZENZO', 'MATSUDAIRA', 'CATHERINE', 'TAMA', 'TOJO', 'OTSU', 'UMIBOZU', 'HANANO', 'KATSUO']
